# ML 2

In [ ]:
#Load python modules : Plot, image , numerical python 
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import numpy as np
from scipy.interpolate import interp1d

# Set a useful font library
font1 = {'size'   : 20, 'family':'STIXGeneral'}
#Colors for plotting 
colors = ['teal', 'yellowgreen', 'gold']
#line width for plottting
lw = 2

# Classification example 2

In [ ]:
# Load some examples to investigate 

exampleno = 1

#Columns: k , P(k) w BAO feature , P(k) w/o BAO feature
fr_exp = np.loadtxt('data/class/fr_mlg/'+str(exampleno)+'.txt')
gr_exp = np.loadtxt('data/class/gr_mlg/'+str(exampleno)+'.txt')


plt.figure(figsize=(12,5))

ax = plt.subplot(1,2,1)
#Pure spectra 
plt.plot(fr_exp[:,0],fr_exp[:,1], color='r')
plt.plot(gr_exp[:,0],gr_exp[:,1], color='b')

ax.tick_params(axis='both', which='major', labelsize=15)
plt.ylabel(' $P(k)$ ',**font1)
plt.xlabel('$k [h/{\\rm Mpc}]$', **font1)

ax = plt.subplot(1,2,2)
#BAO signal 
plt.plot(fr_exp[:,0],fr_exp[:,1]/fr_exp[:,2], color='r', label='f(R)')
plt.plot(gr_exp[:,0],gr_exp[:,1]/gr_exp[:,2], color='b', label='GR')

ax.tick_params(axis='both', which='major', labelsize=15)
plt.ylabel(' BAO signal ',**font1)
plt.xlabel('$k [h/{\\rm Mpc}]$', **font1)
plt.legend(loc='best',prop={'size': 16})


### STEP 1:  Feature selection 

In [ ]:
#Maximum and minimum index to find maximum and minimum of spectrum - our selected feature
maxi = 20
mini = 10

kvalues = fr_exp[mini:maxi,0]
print(kvalues)

#Plot the range over which we are defining our features 
plt.plot(fr_exp[:,0],fr_exp[:,1]/fr_exp[:,2], color='r', label='f(R)')
plt.plot(gr_exp[:,0],gr_exp[:,1]/gr_exp[:,2], color='b', label='GR')

plt.axvline(x=kvalues[0], ymin=0., ymax=10,color='k',linestyle='dotted')
plt.axvline(x=kvalues[-1], ymin=0., ymax=10,color='k',linestyle='dotted')


#### Construct feature vector, test set and cross-validation set 

In [ ]:
# Get number of examples per class 
import os
path, dirs, files = next(os.walk("./data/class/fr_mlg"))
file_count = len(files)

# number of files to be included in test set 
test_set = 101
# number of files to be included in cross-validation set 
cv_set = 201
# Examples to omit
omitted_set = 51

# Final training set size 
m = file_count-test_set-cv_set-omitted_set


# Define an array to store the various values of P(k) at a low k and another at a high k
high_fr = []
high_gr = []
low_fr = []
low_gr = []

# Load the P(k) values per k 
for i in range(m):
    datfr = np.loadtxt('data/class/fr_mlg/'+str(i+1)+'.txt')
    datgr = np.loadtxt('data/class/gr_mlg/'+str(i+1)+'.txt')
    
    #perform rough feature normalisation 
    norm = np.mean(datgr[mini:maxi,1])
    
    # add the features to the feature vector 
    high_fr.append(np.max(datfr[mini:maxi,1]/norm))
    low_fr.append(np.min(datfr[mini:maxi,1]/norm))  
    high_gr.append(np.max(datgr[mini:maxi,1]/norm))
    low_gr.append(np.min(datgr[mini:maxi,1]/norm))


total_low = np.concatenate((low_gr,low_fr), axis=None)
total_high = np.concatenate((high_gr,high_fr), axis=None)


plt.plot(low_fr,high_fr, linestyle='none',marker='x', label='f(R)')
plt.plot(low_gr,high_gr, linestyle='none',marker='o', label='GR')
plt.ylabel('Maximum $P(k)$ ',**font1)
plt.xlabel('Minimum $P(k)$', **font1)
plt.legend(loc='best',prop={'size': 16})


### Let's now fit a decision boundary to this. We will probably need a polynomial boundary ... 

In [ ]:
# Define logistic function first 
def sigmoid(x):
    return 1/(1+np.exp(-x))

xplot = np.linspace(-5,5,100)

plt.plot(xplot,sigmoid(xplot))
plt.axvline(x=0., ymin=0., ymax=10,color='k')
plt.axhline(y=1., xmin=-5, xmax=5,color='k',linestyle='dotted')
plt.axhline(y=0., xmin=-5, xmax=5,color='k',linestyle='dotted')

### Cost function WITH regularisation term 

In [ ]:
#Define our cost function 
# n is the order of the polynomial 
# m is the number of training examples

# x is an (n+2) x m matrix - (1, x2, x1,  x1^n) per example , x1 and x2 are our features (max and min P(k))
# y is an m x 1 matrix - 0 or 1 per example 
# w is an (n+2) x 1 matrix - (bias, w1, w2, ... w(n+2))
# w.T is w transpose 

def mycost(w,x,y,m,lam):
    # define unit matrix
    myunity = np.ones(m)
    
    # Polynomial decision boundary
    mypoly = np.matmul(x.T,w)
        
    # sigmoid of decision boundary curve 
    sigmoidmat = sigmoid(mypoly)   
    
    # the two terms of te binomial cost 
    term1 = np.matmul(y.T,np.log(sigmoidmat))
    term2 = np.matmul((myunity-y).T,np.log(myunity-sigmoidmat))
    
    return -1/m * (term1 + term2) + lam * np.matmul(w,w)

### Set order of polynomial, construct the feature matrix and check cost 

In [ ]:
#function to create feature matrix from x1 and x2 points 
def map_features(low,high, polyorder):
    mylength = len(low)
    bias = np.ones(mylength)
    training_x = [bias]
    for i in range(1,polyorder+1):    
        for j in range(0,i):    
            ind1 = abs(i-j)
            training_x = np.concatenate(( np.array(training_x), [low**ind1 * high**(j)], [high**ind1 * low**(j)] ))
    
    return training_x

In [ ]:
# Create vector of 'right' answers 
training_y = np.concatenate((np.zeros((m)),np.ones((m))))


# Set order of polynomial 
myorder = 8

# Test cost function for random initial guess 
mytrainmat=map_features(total_low,total_high,myorder)

# weight matrix 
winit = np.zeros(len(mytrainmat))

# test the cost function
mycost(winit, mytrainmat , training_y , (m)*2, 0)


### Set regularisation parameter and optimise weights

In [ ]:
# Set regularisation parameter
lam = 0.0

# Use scipy module to optimise the function using a gradient descent algorithm
import scipy.optimize as optimize

xopt = optimize.fmin(mycost, winit , args=(mytrainmat, training_y , m*2, lam), xtol=1e-10, disp=True)

print(xopt)

### Let's see how we did ....

In [ ]:
plt.figure(figsize=(12,5))
ax = plt.subplot(1,2,1)

#Plot decision boundary

# To do that, we need to generate many predictions of X.W  and find the ones that align with 0. 
# We can make a contour plot to do this

# Create a mesh of x and y points from the maximum and minimum of our feature vectors
x_min = total_high.min()
x_max = total_high.max()
y_min = total_low.min()
y_max = total_low.max()

# Steps between min and max 
h=0.001

# Create the mesh 
x_grid, y_grid = np.meshgrid(np.arange(y_min, y_max, h), np.arange(x_min, x_max, h))

# Map these to a feature vector 
myfeature = map_features(x_grid.ravel() ,  y_grid.ravel() , myorder) 

# Get the sigmoid value of these vectors 
Z = sigmoid(np.matmul(myfeature.T,xopt))
# Reshape it for plotting
Z = Z.reshape(x_grid.shape)

#Plot the contour aligning with a sigmoid value of 0.5 (equivalent to X.W=0) 
plt.contour(x_grid, y_grid, Z, levels=[0.5])

plt.plot(low_fr,high_fr, linestyle='none',marker='x', label='f(R)')
plt.plot(low_gr,high_gr, linestyle='none',marker='o', label='GR')

plt.ylabel('Maximum $P(k)$ ',**font1)
plt.xlabel('Minimum $P(k)$', **font1)
plt.legend(loc='best',prop={'size': 16})


ax = plt.subplot(1,2,2)
img = mpimg.imread('data/images/bor3.jpg')
plt.imshow(img)

ax.set_xticks([])
ax.set_yticks([])


### Check cross-validation set accuracy and tune lambda and polynomial order

In [ ]:
# Define a prediction based on our fit 
def predictor(x, w):
    probability = sigmoid(np.matmul(x.T,w))
    return probability
    if(probability >= 0.5):
        return 1
    else:
        return 0

In [ ]:
# Create CV and test data from the unused examples
high_frcv = []
high_grcv = []
low_frcv = []
low_grcv = []

high_frt = []
high_grt = []
low_frt = []
low_grt = []

for i in range(m,m+cv_set-1):
    
    datfr = np.loadtxt('data/class/fr_mlg/'+str(i+1)+'.txt')
    datgr = np.loadtxt('data/class/gr_mlg/'+str(i+1)+'.txt')
    
    #perform rough feature normalisation 
    norm = np.mean(datgr[mini:maxi,1])
    
    high_frcv.append(np.max(datfr[mini:maxi,1]/norm))
    low_frcv.append(np.min(datfr[mini:maxi,1]/norm))  
    high_grcv.append(np.max(datgr[mini:maxi,1]/norm))
    low_grcv.append(np.min(datgr[mini:maxi,1]/norm))

    
for i in range(m+cv_set, m+cv_set+test_set-1):
  
    datfr = np.loadtxt('data/class/fr_mlg/'+str(i+1)+'.txt')
    datgr = np.loadtxt('data/class/gr_mlg/'+str(i+1)+'.txt')

    #perform rough feature normalisation 
    norm = np.mean(datgr[mini:maxi,1])
    
    high_frt.append(np.max(datfr[mini:maxi,1]/norm))
    low_frt.append(np.min(datfr[mini:maxi,1]/norm))  
    high_grt.append(np.max(datgr[mini:maxi,1]/norm))
    low_grt.append(np.min(datgr[mini:maxi,1]/norm))

    
    
total_low_cv = np.concatenate((low_grcv,low_frcv), axis=None)
total_high_cv = np.concatenate((high_grcv,high_frcv), axis=None)

total_low_t = np.concatenate((low_grt,low_frt), axis=None)
total_high_t = np.concatenate((high_grt,high_frt), axis=None)


# Create X.W vector 
cv_feature = map_features(total_low_cv , total_high_cv , myorder) 
t_feature = map_features(total_low_t , total_high_t , myorder) 

# Create vector of 'right' answers 
cv_y = np.concatenate((np.zeros((cv_set-1)),np.ones((cv_set-1))))
t_y = np.concatenate((np.zeros((test_set-1)),np.ones((test_set-1))))


# Calculate the accuracy 
myacccv = 0
myacct = 0

for i in range(cv_set*2-2):
    mypredcv = round(predictor(cv_feature,xopt)[i])

    if(mypredcv==cv_y[i]):
            myacccv +=1 
   

for i in range(test_set*2-2):
    mypredt = round(predictor(t_feature,xopt)[i])
    if(mypredt==t_y[i]):
            myacct +=1 
        
    
print("CV Set accuracy:", myacccv/(cv_set*2-2))
print("Test Set accuracy:", myacct/(test_set*2-2))


# Diagnosing the algorithm

## Let's plot error vs polynomial features 

In [ ]:
# Create arrays for training and cv set costs 

train_costo = []
cv_costo = []

order_array = []

# Set regularisation parameter
lam = 0.0

# Populate the arrays  
for i in range(1,20):
    myorder = i 
    # Training  and CV vectors 
    mytrainmat=map_features(total_low,total_high,myorder)
    mycvmat=map_features(total_low_cv,total_high_cv,myorder)
    
    # initial weight matrix 
    winit = np.zeros(len(mytrainmat))

    xopt = optimize.fmin(mycost, winit , args=(mytrainmat, training_y , m*2, lam), xtol=1e-10, disp=True)
    
    traincost = mycost(xopt, mytrainmat , training_y , m*2, lam)  
    cvcost = mycost(xopt, mycvmat , cv_y , (cv_set-1)*2, lam)  
    
    train_costo.append(traincost)
    cv_costo.append(cvcost)
    order_array.append(i)

### Plot the data

In [ ]:
plt.figure(figsize=(8,8))

#Pure spectra 
plt.plot(order_array ,train_costo, color='b', linestyle='dashed', marker='x', label='Train')
plt.plot(order_array ,cv_costo, color='g',  linestyle='dashed', marker='o',  label= 'Cross-validation')

plt.axvline(x=12, ymin=0., ymax=10,color='k',linestyle='dotted')

ax.tick_params(axis='both', which='major', labelsize=15)
plt.ylabel(' J$(\\theta)$ ',**font1)
plt.xlabel('Order of Polynomial', **font1)

plt.legend(loc='best',prop={'size': 14})


## Plot costs vs lambda

In [ ]:
# Create arrays for training and cv set costs 

train_costl = []
cv_costl = []

lam_array = []

# Set Order
myorder = 5

# Populate the arrays  
for i in range(1,30):
    lam = 0.0 + 0.1*i
    # Training  and CV vectors 
    mytrainmat=map_features(total_low,total_high,myorder)
    mycvmat=map_features(total_low_cv,total_high_cv,myorder)
    
    # initial weight matrix 
    winit = np.zeros(len(mytrainmat))

    xopt = optimize.fmin(mycost, winit , args=(mytrainmat, training_y , m*2, lam), xtol=1e-10, disp=True)
    
    traincost = mycost(xopt, mytrainmat , training_y , m*2, lam)  
    cvcost = mycost(xopt, mycvmat , cv_y , (cv_set-1)*2, lam)  
    
    train_costl.append(traincost)
    cv_costl.append(cvcost)
    lam_array.append(lam)

In [ ]:
plt.figure(figsize=(5,5))

#Pure spectra 
plt.plot(lam_array ,train_costl, color='b', linestyle='dashed', marker='x', label='Train')
plt.plot(lam_array ,cv_costl, color='g',  linestyle='dashed', marker='o',  label= 'Cross-validation')

ax.tick_params(axis='both', which='major', labelsize=15)
plt.ylabel(' J$(\\theta)$ ',**font1)
plt.xlabel('$\lambda$', **font1)

plt.legend(loc='best',prop={'size': 16})


# Assignment: Classify the skewed data set

In [ ]:
# Load some examples to investigate 
exampleno = 2

#Columns: k , P(k) w BAO feature , P(k) w/o BAO feature
fr_exp = np.loadtxt('data/class/skewed/fr/'+str(exampleno)+'.txt')
gr_exp = np.loadtxt('data/class/skewed/gr/'+str(exampleno)+'.txt')


plt.figure(figsize=(12,5))

ax = plt.subplot(1,2,1)
#Pure spectra 
plt.plot(fr_exp[:,0],fr_exp[:,1], color='r')
plt.plot(gr_exp[:,0],gr_exp[:,1], color='b')

ax.tick_params(axis='both', which='major', labelsize=15)
plt.ylabel(' $P(k)$ ',**font1)
plt.xlabel('$k [h/{\\rm Mpc}]$', **font1)

ax = plt.subplot(1,2,2)
#BAO signal 
plt.plot(fr_exp[:,0],fr_exp[:,1]/fr_exp[:,2], color='r', label='f(R)')
plt.plot(gr_exp[:,0],gr_exp[:,1]/gr_exp[:,2], color='b', label='GR')



ax.tick_params(axis='both', which='major', labelsize=15)
plt.ylabel(' BAO signal ',**font1)
plt.xlabel('$k [h/{\\rm Mpc}]$', **font1)
plt.legend(loc='best',prop={'size': 16})


## Feature selection 

In [ ]:
#Maximum and minimum index to find maximum and minimum of spectrum - our selected feature
maxi = 20
mini = 10

kvalues = fr_exp[mini:maxi,0]
print(kvalues)

#Plot the range over which we are defining our features 
plt.plot(fr_exp[:,0],fr_exp[:,1]/fr_exp[:,2], color='r', label='f(R)')
plt.plot(gr_exp[:,0],gr_exp[:,1]/gr_exp[:,2], color='b', label='GR')

plt.axvline(x=kvalues[0], ymin=0., ymax=10,color='k',linestyle='dotted')
plt.axvline(x=kvalues[-1], ymin=0., ymax=10,color='k',linestyle='dotted')


## Plot Skewed data set 

In [ ]:
# Get number of examples per class 
import os
path, dirs, files = next(os.walk("./data/class/skewed/fr"))
file_count = len(files)

pathgr, dirsgr, filesgr = next(os.walk("./data/class/skewed/gr"))
file_countgr = len(filesgr)

# number of files to be included in test set 
test_set = 101
test_setgr = 51

# number of files to be included in cross-validation set 
cv_set = 201
cv_setgr = 151


# F(R) training set size 
m1 = file_count-test_set-cv_set

# GR training set size 
m2 = file_countgr-test_setgr-cv_setgr


# Define an array to store the various values of P(k) at a low k and another at a high k
high_fr = []
high_gr = []
low_fr = []
low_gr = []

# Load the P(k) values per k 
for i in range(m1):
    datfr = np.loadtxt('data/class/fr_mlg/'+str(i+1)+'.txt')
    datgr = np.loadtxt('data/class/gr_mlg/'+str(i+1)+'.txt')
    
    #perform rough feature normalisation 
    norm = np.mean(datgr[mini:maxi,1])
    
    # add the features to the feature vector 
    high_fr.append(np.max(datfr[mini:maxi,1]/norm))
    low_fr.append(np.min(datfr[mini:maxi,1]/norm))  

for i in range(m2):

    datgr = np.loadtxt('data/class/gr_mlg/'+str(i+1)+'.txt')
   
    norm = np.mean(datgr[mini:maxi,1])

    high_gr.append(np.max(datgr[mini:maxi,1]/norm))
    low_gr.append(np.min(datgr[mini:maxi,1]/norm))
    
    
total_low = np.concatenate((low_gr,low_fr), axis=None)
total_high = np.concatenate((high_gr,high_fr), axis=None)


plt.plot(low_fr,high_fr, linestyle='none',marker='x', label='f(R)')
plt.plot(low_gr,high_gr, linestyle='none',marker='o', label='GR')
plt.ylabel('Maximum $P(k)$ ',**font1)
plt.xlabel('Minimum $P(k)$', **font1)
plt.legend(loc='best',prop={'size': 16})
